#### Financial Market Data Pre-Processor

##### Software Pre-requisites:
```
pip install pandas
pip install finnhub-python
```

##### Running instructions 
- Press `Run All` Button

In [17]:

import os
import time
import finnhub
import pandas as pd
import base64
from pathlib import Path
from datetime import date, timedelta

# dates for request parameters
start_time = time.time()
date_today = date.today()
today_date = date_today.strftime('%Y-%m-%d')
current_year = str(date_today.year)

from_time_unix = int(time.mktime((date_today - timedelta(weeks = 52)).timetuple()))
to_time_unix = int(time.mktime(date_today.timetuple()))


In [18]:
# Finnhub clients setup - https://finnhub.io/docs/api/
def generate_finnhub_clients():
    finnhub_keys = ['Y2Q0b3FpYWFkM2k5OGpodTJwZ2djZDRvcWlhYWQzaTk4amh1MnBoMA==','Y2UzbWQyYWFkM2kxaDJuN24xODBjZTNtZDJhYWQzaTFoMm43bjE4Zw==','Y2R0ZG1yMmFkM2k0MXY3aG9nM2djZHRkbXIyYWQzaTQxdjdob2c0MA==',
        'Y2U1NmViaWFkM2lmZHZ0aHQzcjBjZTU2ZWJpYWQzaWZkdnRodDNyZw==','Y2U0dTFiMmFkM2llMTg4dGY4bmdjZTR1MWIyYWQzaWUxODh0ZjhvMA==','Y2UzbWU2MmFkM2kxaDJuN24xc2djZTNtZTYyYWQzaTFoMm43bjF0MA==',
        'Y2UzbWtiYWFkM2kxaDJuN240ZGdjZTNta2JhYWQzaTFoMm43bjRlMA==','Y2U1NjlxMmFkM2lmZHZ0aHQwZTBjZTU2OXEyYWQzaWZkdnRodDBlZw==','Y2UzbXYxYWFkM2kxaDJuN244dWdjZTNtdjFhYWQzaTFoMm43bjh2MA==',
        'Y2U0dHE5YWFkM2llMTg4dGY0YzBjZTR0cTlhYWQzaWUxODh0ZjRjZw==','Y2U0dTQyaWFkM2llMTg4dGZhYWdjZTR1NDJpYWQzaWUxODh0ZmFiMA==','Y2U1NmQ2cWFkM2lmZHZ0aHQzM2djZTU2ZDZxYWQzaWZkdnRodDM0MA==']

    finnhub_client_list = []

    for key in finnhub_keys:
        finnhub_client_list.append(finnhub.Client(api_key=base64.b64decode(key).decode()))

    return finnhub_client_list

finnhub_clients = generate_finnhub_clients()
client_num = 0

def get_finhub_client():
    """ returns a finnhub client to perform requests to gather financial data """

    global client_num
    if client_num >= len(finnhub_clients) - 1:
        client_num = 0
    else:
        client_num+=1

    return finnhub_clients[client_num]

In [19]:
def create_csv_path(csv_path):
    csv_file_path = Path(csv_path)
    csv_file_path.parent.mkdir(parents=True, exist_ok=True)
    return csv_file_path


In [20]:
def retrieve_candlestick_data_frame(ticker_symbol):
    
    candle_response = get_finhub_client().stock_candles(symbol=ticker_symbol, resolution='D', _from=from_time_unix, to=to_time_unix)

    candles = pd.json_normalize(candle_response)[['t', 'o', 'c', 'h', 'l', 'v']]
    
    candles_df = pd.DataFrame(columns=['date','unix_time', 'open', 'close', 'high', 'low', 'volume'])
    
    candles_df['unix_time'] = candles['t'][0]
    candles_df['open'] = candles['o'][0]
    candles_df['close'] = candles['c'][0]
    candles_df['high'] = candles['h'][0]
    candles_df['low'] = candles['l'][0]
    candles_df['volume'] = candles['v'][0]

    candles_df.sort_values(by=['unix_time'], ascending=False, inplace=True)
    candles_df.drop_duplicates(subset=['unix_time'], keep='last', inplace=True)

    candles_df['date'] = pd.to_datetime(candles_df['unix_time'],unit='s').astype(str)    

    return candles_df


In [21]:
def populate_candlestick_data(ticker_symbol):
    """ populates the daily candlestick data for the given stock into the file: 'data/candlestick_data.csv' """

    candlestick_df = retrieve_candlestick_data_frame(ticker_symbol)

    if not(candlestick_df.empty):

        candlestick_df.insert(0,'symbol', ticker_symbol)

        csv_file_path = create_csv_path("data/candlestick_data.csv")

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, candlestick_df], axis=0).drop_duplicates(subset=['symbol', 'unix_time'], keep='last').sort_values(by=['symbol', 'unix_time'], ascending=[True, False])
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                candlestick_df.sort_values(by=['symbol', 'unix_time'], ascending=[True, False]).to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while generating candlestick data for {ticker_symbol}, error: {error}")


In [22]:

def retrieve_technical_data_frame(ticker_symbol):
    
    bband_response = get_finhub_client().technical_indicator(symbol=ticker_symbol, resolution='D', _from=from_time_unix, to=to_time_unix, indicator='bbands', indicator_fields={"timeperiod": 20})
    tech_data = pd.json_normalize(bband_response)[['t', 'o', 'c', 'h', 'l', 'v', 'lowerband', 'middleband', 'upperband']]
    
    technical_df = pd.DataFrame(columns=['date','unix_time', 'open', 'close', 'high', 'low', 'volume', 'lowerband', 'middleband', 'upperband'])
    
    technical_df['unix_time'] = tech_data['t'][0]
    technical_df['open'] = tech_data['o'][0]
    technical_df['close'] = tech_data['c'][0]
    technical_df['high'] = tech_data['h'][0]
    technical_df['low'] = tech_data['l'][0]
    technical_df['volume'] = tech_data['v'][0]
    technical_df['lowerband'] = tech_data['lowerband'][0]
    technical_df['middleband'] = tech_data['middleband'][0]
    technical_df['upperband'] = tech_data['upperband'][0]

    technical_df.sort_values(by=['unix_time'], ascending=False, inplace=True)
    technical_df.drop_duplicates(subset=['unix_time'], keep='last', inplace=True)

    technical_df = technical_df[technical_df['upperband'] > 0]

    technical_df['date'] = pd.to_datetime(technical_df['unix_time'],unit='s').astype(str)

    return technical_df

In [23]:
def populate_technical_data(ticker_symbol):
    """ populates the daily technical indicator data for the given stock into the file: 'data/technical_indicators.csv' """

    stock_technical_df = retrieve_technical_data_frame(ticker_symbol)

    if not(stock_technical_df.empty):

        stock_technical_df.insert(0,'symbol', ticker_symbol)

        csv_file_path = create_csv_path("data/technical_indicators.csv")

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, stock_technical_df], axis=0).drop_duplicates(subset=['symbol', 'unix_time'], keep='last').sort_values(by=['symbol', 'unix_time'], ascending=[True, False])
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                stock_technical_df.sort_values(by=['symbol', 'unix_time'], ascending=[True, False]).to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while generating technical indicator data for {ticker_symbol}, error: {error}")


In [24]:
renamed_social_columns = {"mention": "mention_twitter", "positiveScore": "positiveScore_twitter","negativeScore": "negativeScore_twitter",
    "positiveMention": "positiveMention_twitter","negativeMention": "negativeMention_twitter","score": "score_twitter"}

def retrieve_social_sentiment_data_frame(ticker_symbol):

    social_response = get_finhub_client().stock_social_sentiment(ticker_symbol)

    twitter_social_df = pd.json_normalize(social_response, record_path='twitter')
    reddit_social_df = pd.json_normalize(social_response, record_path='reddit')

    try:
        if 'atTime' in reddit_social_df:
            social_df = twitter_social_df.merge(reddit_social_df, how='left', on=['atTime'], suffixes=('', '_reddit')).fillna(0).rename(columns=renamed_social_columns)

        else:
            social_df = twitter_social_df.fillna(0).rename(columns=renamed_social_columns)

    except Exception as error:
        print(f"Error while gathering social sentiment for {ticker_symbol} error: {error}")
        social_df = twitter_social_df.fillna(0).rename(columns=renamed_social_columns)

    return social_df

In [25]:
def populate_social_sentiment(ticker_symbol):
    """ populates the social sentiment for stocks on Reddit and Twitter for the given stock into the file: 'data/social_media_sentiment.csv' """

    social_df = retrieve_social_sentiment_data_frame(ticker_symbol)
    if not(social_df.empty):
        
        social_df.insert(0,'symbol', ticker_symbol)
        csv_file_path = create_csv_path("data/social_media_sentiment.csv")

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, social_df], axis=0).drop_duplicates(keep='last').sort_values(by=['symbol', 'atTime'], ascending=[True, False]).fillna(0)
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                social_df.to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while generating social sentiment data for {ticker_symbol}, error: {error}")



In [26]:
insider_trans_cols =['symbol','share','change','transactionDate','transactionCode','transactionPrice','name','filingDate','id']

def populate_insider_transactions(ticker_symbol):
    """ populates insider transactions into file 'data/insider_transactions.csv' """

    insider_transactions_response = get_finhub_client().stock_insider_transactions(ticker_symbol)

    insider_trans_df = pd.json_normalize(insider_transactions_response, record_path='data')

    if not insider_trans_df.empty:
        csv_file_path = create_csv_path("data/insider_transactions.csv")

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, insider_trans_df], axis=0).drop_duplicates().sort_values(by=['symbol','transactionDate'], ascending=[True, False])[insider_trans_cols]
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                insider_trans_df.sort_values(by=['symbol', 'transactionDate'], ascending=[True, False])[insider_trans_cols].to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while generating insider transactions data for {ticker_symbol}, error: {error}")

In [27]:
earnings_cols =['symbol','period','actual','estimate','surprise','surprisePercent']

def populate_company_surprise_earnings(ticker_symbol):
    """ populates company surprise earnings into file 'data/surprise_earnings.csv' """

    earnings_response = get_finhub_client().company_earnings(ticker_symbol)

    earnings_df = pd.json_normalize(earnings_response)

    if not earnings_df.empty:
        csv_file_path = create_csv_path("data/surprise_earnings.csv")

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, earnings_df], axis=0).drop_duplicates(subset=['symbol', 'period']).sort_values(by=['symbol','period'], ascending=[True, False])[earnings_cols]
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                earnings_df.sort_values(by=['symbol', 'period'], ascending=[True, False])[earnings_cols].to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while generating company earnings data for {ticker_symbol}, error: {error}")

In [28]:
def populate_insider_sentiment(ticker_symbol):
    """ populates the insider sentiment data into the file: 'data/insider_sentiment.csv' """

    insider_response = get_finhub_client().stock_insider_sentiment(ticker_symbol, "2013-01-01", current_year+"-12-31")

    insider_df = pd.json_normalize(insider_response, record_path='data')

    if not insider_df.empty:
        csv_file_path = create_csv_path("data/insider_sentiment.csv")

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, insider_df], axis=0).drop_duplicates().sort_values(by=['symbol','year','month'], ascending=[True,False,False])
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                insider_df.sort_values(by=['symbol','year','month'], ascending=[True,False,False]).to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while generating insider sentiment data for {ticker_symbol}, error: {error}")
    

In [29]:
trends_columns = ['symbol', 'period', 'strongBuy', 'buy', 'hold', 'sell', 'strongSell']

def populate_recommended_trends(ticker_symbol):
    """ populates the latest analyst recommendation trends for a company into the file: 'data/recommendation_trends.csv' """

    trends = get_finhub_client().recommendation_trends(symbol=ticker_symbol)

    trends_df = pd.json_normalize(trends)

    if 'symbol' in trends_df and len(trends_df['symbol']) > 0:

        trends_df[trends_columns]
        csv_file_path = create_csv_path("data/recommendation_trends.csv")

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, trends_df], axis=0).sort_values(by=['symbol', 'period'], ascending=[True, False]).drop_duplicates(subset=['symbol', 'period'])[trends_columns]
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                trends_df.sort_values(by=['symbol', 'period'], ascending=[True, False]).to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while updating recommendation trends data for {ticker_symbol}, error: {error}")


In [30]:
def populate_senate_lobbying(ticker_symbol):
    """ populates the reported lobbying activities in the Senate and the House into the file: 'data/senate_lobbying.csv' """

    lobby_response = get_finhub_client().stock_lobbying(ticker_symbol, "2000-01-01", today_date)

    lobby_df = pd.json_normalize(lobby_response, record_path='data')

    if not lobby_df.empty:
        csv_file_path = create_csv_path("data/senate_lobbying.csv")

        symbol_col = lobby_df.pop('symbol')
        year_col = lobby_df.pop('year')
        lobby_df.insert(0, 'symbol', symbol_col)
        lobby_df.insert(0, 'year', year_col)

        try:
            if os.path.exists(csv_file_path):
                existing_df = pd.read_csv(csv_file_path, header = 0)
                updated_df = pd.concat([existing_df, lobby_df], axis=0).sort_values(by=['symbol', 'year'], ascending=[True, False]).drop_duplicates().dropna(how='all', axis=1)
                updated_df.to_csv(csv_file_path, encoding='utf-8', index=False)

            else:
                lobby_df.dropna(how='all', axis=1).to_csv(csv_file_path, encoding='utf-8', index=False)

        except Exception as error:
            print(f"Error while generating senate lobbying data for {ticker_symbol}, error: {error}")


In [31]:
def should_generate_data_for_ticker(stock_info, stock_tickers):
    return 'symbol' in stock_info and len(stock_info['symbol']) > 0 and stock_info['symbol'] in stock_tickers

In [32]:
# Hardcoded set of symbols to generate data. Includes Comcast competitors and companies from the sectors listed below:
# Ride Share, Gambling, Sports Entertainmnet, Big Tech, Social Media
stock_tickers = {'AAPL','AMZN','AMC','AMD','ANGI','ATUS','BAC','BETZ','BMBL','BYD','CABO','CHDN','CHTR','CMCSA','CRM','CVX','CZR','DIS','DISH','DKNG','DG','EA','EFX',
'F','FUBO','FWONA','GOOG','GME','HD','INTC','JNJ','JPN','KO','LBRDA','LUMN','LLY','LSXMA','LVS','LYFT','MANU','META','MA','MGM','MRK','MSFT','MSGS','MTCH','MTN',
'NFLX','NKE','NVDA','NWSA','PARA','PENN','PEP','PDYPY','PFE','PG','PINS','RCI','ROKU','SAVE','SIRI','SNAP','SOFI','SPOT','T','TLSA','TVTV','TWTR','UBER','UNH',
'V','VZ','WBD','WFC','WOW','WWE','WYNN','XOM','YELP'}

all_stock_info = get_finhub_client().stock_symbols(exchange="US", currency="USD", security_type="Common Stock")
processed_symbols = []

print("Please Wait... Generating Financial Data. Interrupt the Program to Exit")

for stock_info in all_stock_info:

    if should_generate_data_for_ticker(stock_info, stock_tickers):
        ticker_symbol = stock_info['symbol']

        try:

            populate_candlestick_data(ticker_symbol)

            populate_technical_data(ticker_symbol)

            populate_social_sentiment(ticker_symbol)

            populate_insider_transactions(ticker_symbol)

            #populate_insider_sentiment(ticker_symbol)

            #populate_recommended_trends(ticker_symbol)

            #populate_company_surprise_earnings(ticker_symbol)

            #populate_senate_lobbying(ticker_symbol)

            processed_symbols.append(ticker_symbol)

            if len(processed_symbols) % 5 == 0:
                print(f"{len(processed_symbols)} Symbols Processed. Current Run Time: {(time.time() - start_time)} seconds")

        except Exception as e:
            print(f"Error while generating data for {ticker_symbol}, error: {e}")
            time.sleep(5)
        except KeyboardInterrupt:
            print("Process Interrupted")
            break

print(f"-- Exiting Program -- Total Execution Time: {(time.time() - start_time) / 60} minutes")
print(f"{len(processed_symbols)} Stock Symbols Processed. Symbol List: {processed_symbols}")

Please Wait... Generating Financial Data. Interrupt the Program to Exit
5 Symbols Processed. Current Run Time: 5.020374059677124 seconds
10 Symbols Processed. Current Run Time: 7.435729026794434 seconds
15 Symbols Processed. Current Run Time: 10.480255126953125 seconds
20 Symbols Processed. Current Run Time: 12.655170917510986 seconds
25 Symbols Processed. Current Run Time: 14.660192966461182 seconds
30 Symbols Processed. Current Run Time: 16.241300106048584 seconds
35 Symbols Processed. Current Run Time: 20.56233525276184 seconds
40 Symbols Processed. Current Run Time: 23.230959177017212 seconds
45 Symbols Processed. Current Run Time: 24.93332004547119 seconds
50 Symbols Processed. Current Run Time: 27.40036916732788 seconds
55 Symbols Processed. Current Run Time: 30.35839605331421 seconds
60 Symbols Processed. Current Run Time: 32.93816113471985 seconds
65 Symbols Processed. Current Run Time: 35.846702098846436 seconds
70 Symbols Processed. Current Run Time: 39.397249937057495 second